In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LeNet5_CIFAR10
import tf_general as tfg
import numpy as np
import aws_boto3

FLAGS = tf.flags.FLAGS

try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 20000, 'epoch')
    tf.flags.DEFINE_integer('batch_size',200, 'batch size')
    tf.flags.DEFINE_integer('test_size', 200, 'test size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_float('keep_prob', 0.5, 'keep prob for drop lay')
    tf.flags.DEFINE_boolean('augument', True, 'if image augument is applied')
                            
    tf.flags.DEFINE_float('ckpt_frequency', 250, 'frequency to save checkpoint')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

print("learn Rate =",FLAGS.lr, "keep Prob=", FLAGS.keep_prob, " image augument=",FLAGS.augument)   
CONTINUE = 0

parameters were defined.
learn Rate = 0.01 keep Prob= 0.5  image augument= True


In [2]:
data = cifar10();
train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size)

train set length: 50000
test image shape: (10000, 3072)
test label shape: (10000,)
test set length: 10000


In [3]:
ckpt_dir = 'ckpt_0/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None,32,32,3], name='x_input')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    y_ = tf.placeholder(tf.int64, [None], name='labels')

In [5]:
with tf.name_scope('prediction'):
    le_net5 = LeNet5_CIFAR10(x, keep_prob)
    y = le_net5.prediction

conv_1:  (?, 28, 28, 18)
pool_1:  (?, 14, 14, 18)
conv_2:  (?, 10, 10, 48)
pool_2:  (?, 5, 5, 48)
flat_1: (?, 1200)
fc_1:  (?, 120)
fc_2:  (?, 84)
fc_3:  (?, 10)
drop_out_1:  (?, 10)
prediction:  (?, 10)


In [6]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                  labels=y_, 
                                                                                  name="cross_entropy_per_example"))

In [7]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [8]:
prediction =tf.argmax(y, 1)
accuracy = tf.reduce_mean(tf.cast( tf.equal(prediction,y_), tf.float32))

In [9]:
def SaveCheckpoint2S3():
    bucketname = 'sagemaker-deechean-dl' 
    path = 'lenet_ckpt/'
    for file in os.listdir('ckpt'):
        if os.path.isfile('ckpt/'+file):
            print('save files to s3')
            aws_boto3.upload_file('ckpt/'+file, bucketname,path+file)

In [10]:
import time
filename = 'lenet_train_cifar10_0.log'

if __name__ == '__main__':
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep=1)
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint(ckpt_dir)
            saver.restore(sess,model_file)
        for i in range(CONTINUE, FLAGS.epoch):
            train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size,FLAGS.augument)
            loss, _,accuracy_rate = sess.run([cross_entropy, train_step,accuracy], 
                                             feed_dict={keep_prob: FLAGS.keep_prob, x:train_image, y_:train_label})
            tfg.savelog(filename,'train_accuracy',i, accuracy_rate)  
            tfg.savelog(filename,'train_loss',i, loss)   
            if (i+1) % FLAGS.ckpt_frequency == 0:  #保存预测模型
                saver.save(sess,ckpt_dir+'cifar10_'+str(i+1)+'.ckpt',global_step=i+1)
                #SaveCheckpoint2S3()
                acc_accuracy = 0
                for j in range(int(10000/FLAGS.test_size)):                    
                    test_image, test_label,test_index = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate, output = sess.run([accuracy,prediction],
                                                     feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                    acc_accuracy += accuracy_rate
                    tfg.savelog(filename,'test_index',i, test_index)
                    tfg.savelog(filename,'output',i, output)
                accuracy_rate = acc_accuracy/10000*FLAGS.test_size
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + 
                      ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%')
                tfg.savelog(filename,'test_accuracy',i, accuracy_rate)            
    tf.reset_default_graph()

ValueError: axes don't match array